In [ ]:
import numpy as np
import pandas as pd
import pathlib
import time
import matplotlib.pyplot as plt

In [ ]:
# Set path
path = pathlib.Path(r"C:\Users\Mathiass\OneDrive - Universität Zürich UZH\Documents\mt_literature\data")

In [ ]:
# read dataset
data = pd.read_parquet(path/"final_df_filledmean_small.parquet")

In [ ]:
# # # create smaller dataset to parquet
# X_1 = data.iloc[:, :19]
# X_2 = data[["cp_flag_C", "cp_flag_P"]]
# y = data["option_ret"]
# test = pd.concat([X_1, X_2, y], axis=1)
# test.to_parquet(fr"{path}\final_df_filledmean_small.parquet")

In [ ]:
# FEATURE ENGINEERING
# Bid-Ask-Spread
data["best_bid"] = (data["best_offer"] - data["best_bid"]) / (data["best_offer"])
data = data.rename(columns={"best_bid": "ba_spread_option"}).drop(["best_offer"], axis=1)

# Gamma -> multiply by spotprice and divide by 100
data["gamma"] = data["gamma"] * data["spotprice"] / 100 #following Bali et al. (2021)

# Theta -> scale by spotprice
data["theta"] = data["theta"] / data["spotprice"] #following Bali et al. (2021)

# Vega -> scale by spotprice
data["vega"] = data["vega"] / data["spotprice"] #following Bali et al. (2021)

# Time to Maturity -> scale by number of days in year -> 365
data["days_to_exp"] = data["days_to_exp"] / 365

# Moneyness
data["strike_price"] = data["strike_price"] / data["spotprice"] # K / S
data = data.rename(columns={"strike_price": "moneyness"})

# Forward Price ratio
data["forwardprice"] = data["forwardprice"] / data["spotprice"]

# drop cf_adj + days no trading
data = data.drop(["cfadj", "days_no_trading", "spotprice", "adj_spot"], axis=1)


In [ ]:
data.describe()

In [ ]:
# # multiclass y label function
# def binary_categorize(x):
#     if x > 0:
#         return 1
#     else:
#         return 0

# # apply label function to option returns
# data["option_ret"] = data["option_ret"].apply(binary_categorize)

# data[data["cp_flag_C"] == 0]["option_ret"].plot(kind="hist")

# data[data["cp_flag_C"] == 1]["option_ret"].plot(kind="hist")

# data["option_ret"].plot(kind="hist")

# def percent_one(x):
#     return np.sum(x) / len(x)

# percent_one(data[data["cp_flag_C"] == 1]["option_ret"])

# percent_one(data[data["cp_flag_P"] == 1]["option_ret"])

In [ ]:
# drop large outlier

# to_drop = []


# to_drop += (list(np.where(data["moneyness"] > np.percentile(data["moneyness"], 99))[0]))
# to_drop += (list(np.where(data["ba_spread_option"] > np.percentile(data["ba_spread_option"], 99))[0]))
# to_drop += (list(np.where(data["volume"] > np.percentile(data["volume"], 99))[0]))
# to_drop += (list(np.where(data["open_interest"] > np.percentile(data["open_interest"], 99))[0]))

# data = data.drop(set(to_drop)).reset_index(drop=True)

In [ ]:
data

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
tscv = TimeSeriesSplit(n_splits=17)

In [ ]:
for train, test in tscv.split(data):
    print("%s %s" % (train, test))

In [ ]:
data[:212415]

In [ ]:
# # DROP MONEYNESS BELOW 0.8 AND ABOVE 1.2
# to_drop = []


# to_drop += (list(np.where((data["moneyness"] > 1.2) | (data["moneyness"] < 0.8))[0]))
# data = data.drop(set(to_drop)).reset_index(drop=True)

In [ ]:
# # take log of features where distribution is skewed
# # data["moneyness"] = np.log(data["moneyness"])
# data["ba_spread_option"] = np.log(data["ba_spread_option"])
# data["volume"] = np.log(data["volume"])
# data["open_interest"] = np.log(data["open_interest"])
# data["impl_volatility"] = np.log(data["impl_volatility"])
# # data["days_to_exp"] = np.log(data["days_to_exp"])
# data["mid_price"] = np.log(data["mid_price"])

# # data["forwardprice"] = np.log(data["forwardprice"])
# # data["ir_rate"] = np.log(data["ir_rate"])

In [ ]:
data

In [ ]:
np.where(data["ba_spread_option"] > np.percentile(data["ba_spread_option"], 99))[0]

In [ ]:
# Plot each column
for i in data.columns:
    plt.figure()
    plt.hist(data[i], bins=20)
    plt.title(i)

In [ ]:
np.log(data["days_to_exp"]).plot(kind="hist", bins=50)

In [ ]:
data.drop(np.where(data["ba_spread_option"] > np.percentile(data["ba_spread_option"], 99))[0])

In [ ]:
data

In [ ]:
# take out outliers
np.where(data["moneyness"] > np.percentile(data["moneyness"], 99))[0]

In [ ]:
# drop outliers
data[data["open_interest"] > np.percentile(data["open_interest"], 99)]

In [ ]:
IQA = np.percentile(data["ba_spread_option"], 75) - np.percentile(data["ba_spread_option"], 25)

In [ ]:
data[data["ba_spread_option"] > 5 * IQA]

In [ ]:
np.percentile(data["ba_spread_option"], 99.99)

In [ ]:
data[data["ba_spread_option"] > np.percentile(data["ba_spread_option"], 99.99)]

In [ ]:
data["theta"].describe()

In [ ]:
np.log(data["days_to_exp"]).plot(kind="hist")

In [ ]:
data["moneyness"].describe()

In [ ]:
np.percentile(data["moneyness"], 90)

In [ ]:
data[data["moneyness"] > 40]

In [ ]:
data.describe()

In [ ]:
# def standardize(x):
#     return (x - np.mean(x)) / np.std(x)

# # Plot each column
# for i in data.columns[1:]:
#     plt.figure()
#     plt.hist(standardize(data[i]))
#     plt.title(i)

In [ ]:
data[(data["option_ret"] > -0.5) & (data["option_ret"] < 2)]

In [ ]:
# -> moved to data prep file
# data = data.drop(["secid", "optionid", "date", "exdate", "sdate", "edate", "permno"], axis=1)
# -> moved to data prep file. sort values by data! (optionally by secid -> but better to shuffle later)
# data = data.sort_values(["date", "secid"]).reset_index(drop=True)

In [ ]:
# How many % missing values do we have?
np.sum(np.sum(data.isnull())) / (data.shape[0] * data.shape[1]) * 100

In [ ]:
# takes a few mins
# data.corr()

In [ ]:
# create X and y datasets
y = data["option_ret"]
# drop it for X
X = data.drop(["option_ret"], axis=1)

In [ ]:
len(y[y > 0])

In [ ]:
1393098 / len(y)

In [ ]:
# multiclass y label function
def binary_categorize(y):
    if y > 0:
        return 1
    else:
        return 0

In [ ]:
# multiclass y label function
def multi_categorize(y):
    if y > 0.05:
        return 1
    elif y < -0.05:
        return -1
    else:
        return 0

In [ ]:
# apply label function to option returns
y = y.apply(multi_categorize)

In [ ]:
# from sklearn.model_selection import train_test_split

In [ ]:
np.min(X["theta"])

In [ ]:
# # Split into train, test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False, random_state=0)
# # Split train into train and val
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, testsize=0.4, shuffle=False, random_state=0)

In [ ]:
# define start years for validation and test periods
# start_train = 1996 (fixed)
start_val = "2015"
start_test = "2016"

In [ ]:
# extract and save dates column -> needed to sort into train, val, test
dates = X["date"]
X = X.drop(["date"], axis=1)

In [ ]:
# Create train, val, test sets & convert to numpy
# Train
X_train = X[dates < start_val].values
y_train = y[:len(X_train)].values

In [ ]:
# Val
mask = (dates >= start_val) & (dates < start_test)
X_val = X[mask].values
y_val = y[len(X_train):len(X_train)+len(X_val)].values

In [ ]:
# Test
X_test = X[dates >= start_test].values
y_test = y[-len(X_test):].values

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [ ]:
# ONLY SCALE NON-DUMMY CONTINOUS VARIABLES
# scaler = ColumnTransformer(
#     [("scaler", StandardScaler(), slice(None, -2))],
#     remainder="passthrough"
# )

In [ ]:
scaler = StandardScaler()
X_train_pca = scaler.fit_transform(X_train)

In [ ]:
pca = PCA(n_components=10)

In [ ]:
pca.fit(X_train_pca)

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("number of components")
plt.ylabel("cumulative expl. variance")

In [ ]:
X_train.shape

In [ ]:
pca = PCA(n_components=10)

In [ ]:
scaler = StandardScaler()

In [ ]:
clf = LogisticRegression(random_state=0, 
                         class_weight="balanced",
                           max_iter=1000,
#                          n_jobs=-1,
#                          C=0.0001,
                        )

# clf = RandomForestClassifier(random_state=0,
#                     class_weight="balanced",
#                     max_depth=2,
#                     n_jobs=-1,
#                 )

# clf = HistGradientBoostingClassifier(random_state=0,
#                                  max_iter=100000, 
#                                  max_depth=2,
# #                                  learning_rate=1.0,
#                                 validation_fraction=None,
#                                  verbose=1,
#                                 )

# clf = LinearSVC(random_state=0,
#                class_weight="balanced",
#                )

# clf = SVC(random_state=0,
#          class_weight="balanced",
#          )

In [ ]:
len(X_train)

In [ ]:
clf = Pipeline([
    ('scaler', scaler),
#     ('pca' , pca),
    ('clf', clf),
])

In [ ]:
# np.logspace(-2, 2, 5)

In [ ]:
# train_indices = [[1,3,5,7,9],[2,4,6,8]]
# test_indices = [[2,4,6,8],[1,3,5,7,9]]

In [ ]:
# a = zip(train_indices, test_indices)

In [ ]:
# for i, y in a:
#     print(i, y)

In [ ]:
# def cv_gen():
#     for i in range(3):
#         yield i, i*2

In [ ]:
param_grid = {
#     "clf__C": np.logspace(-15, 0, 5),
#       "clf__max_depth": [2, 3, 4],
}
clf = GridSearchCV(
    clf, 
    param_grid,
    scoring=["accuracy", "balanced_accuracy"],
    refit="balanced_accuracy",
    n_jobs=-1,
#     cv=a,
    verbose=3,
)

In [ ]:
# sample_weight
# sample_weight = 3 * (len(y_train) - np.sum(y_train)) / np.sum(y_train)

In [ ]:
s_time = time.time()
clf.fit(X_train, y_train,
#        clf__sample_weight=sample_weight
       )
e_time = time.time()
print("Time to fit: ", (e_time - s_time), "seconds")

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

In [ ]:
y_trivial = np.zeros_like(y_val)

In [ ]:
pd.value_counts(y_val)

In [ ]:
np.mean(y_val)

##### Trivial prediction

In [ ]:
# TRIVIAL prediction
accuracy_score(y_val, y_trivial)

In [ ]:
# TRIVIAL prediction
balanced_accuracy_score(y_val, y_trivial)

#### Classifier prediction

In [ ]:
# CLASSIFIER
y_pred = clf.predict(X_val)

In [ ]:
# CLASSIFIER
accuracy_score(y_val, y_pred)

In [ ]:
# 0.54

In [ ]:
# CLASSIFIER
balanced_accuracy_score(y_val, y_pred)

In [ ]:
# LR 0.5240759984109695
#RF 0.5263639414267005

In [ ]:
# 0.52

In [ ]:
pd.value_counts(y_pred)

In [ ]:
np.mean(y_pred)

In [ ]:
np.sum(y_pred) / len(y_pred)

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(X_train, y_train)

In [ ]:
y_dummy = dummy_clf.predict(X_val)

In [ ]:
accuracy_score(y_val, y_dummy)

In [ ]:
balanced_accuracy_score(y_val, y_dummy)

In [ ]:
clf.cv_results_


In [ ]:
clf.best_params_